In [23]:
import pinecone
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv

load_dotenv()

True

In [24]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

In [3]:
def load_pdf(data):
  loader = DirectoryLoader(
    data,
    glob="*.pdf",
    loader_cls=PyPDFLoader
  )

  documents = loader.load()

  return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
def split_text_into_chunks(extracted_data):
  text_splitter =RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
  text_chunks = text_splitter.split_documents(extracted_data)

  return text_chunks

In [6]:
text_chunks = split_text_into_chunks(extracted_data)
print("length of text_chunks: ", len(text_chunks))

length of text_chunks:  7020


In [7]:
# This will download the model from Hugging Face and cache it locally
def hugging_face_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings

In [8]:
embeddings = hugging_face_embeddings()

In [28]:
pc = pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

docsearch = Pinecone.from_texts(
  [t.page_content for t in text_chunks],
  embeddings,
  index_name=INDEX_NAME,
)

In [40]:
docsearch = Pinecone.from_existing_index(embedding=embeddings, index_name=INDEX_NAME)

query = "What are Allergies"

docs = docsearch.search(query, k=3, search_type="similarity")

print("Results: ", docs)

Results:  [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of d

In [41]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [42]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = { "prompt": PROMPT }

In [43]:
llm=CTransformers(
  model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
  model_type="llama",
  config={'max_new_tokens':512,
  'temperature':0.8}
)

In [45]:
qa=RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
  return_source_documents=True,
  chain_type_kwargs=chain_type_kwargs
)

In [48]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response: ", result["result"])

Response:  Acne is a common skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria. To avoid acne, it's important to keep your skin clean by washing your face regularly, especially after exercising or sweating. Avoid picking or squeezing blemishes, as this can cause further inflammation and scarring. Eating a well-balanced diet that is rich in fruits, vegetables, whole grains, and lean protein can also help to reduce the severity of acne. Stress can also exacerbate acne, so finding ways to manage stress, such as through exercise or meditation, can be helpful.
